In [190]:
!pip install pandas

In [210]:
import json
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import os
import unicodeblock.blocks
import re
from tqdm import tqdm
from random import random

In [211]:
links = {}
expected_unicodes = {}

with open('language-pairs.json', 'r') as f:
    pairs = json.loads(f.read())
    
    for pair in pairs:
        expected_unicodes[pair] = pairs[pair]['source']['unicode']
        links[pair] = pairs[pair]['wiki']

links, expected_unicodes

({'Assamese-Sanskrit': 'https://en.m.wiktionary.org/wiki/Category:Assamese_terms_borrowed_from_Sanskrit',
  'Bengali-Sanskrit': 'https://en.m.wiktionary.org/wiki/Category:Bengali_terms_borrowed_from_Sanskrit'},
 {'Assamese-Sanskrit': 'Devanagari', 'Bengali-Sanskrit': 'Devanagari'})

In [212]:
# expected Unicode blocks for certain scripts
unicode_block_map = {
    "Latin" : ['BASIC_LATIN', 'LATIN_1_SUPPLEMENT', 'LATIN_EXTENDED_LETTER', 'LATIN_EXTENDED_A', 'LATIN_EXTENDED_B',\
               'LATIN_EXTENDED_C'],
    "Greek" : ['GREEK'],
    "Cyrillic" : ['CYRILLIC', 'CYRILLIC_SUPPLEMENTARY', 'CYRILLIC_EXTENDED_A', 'CYRILLIC_EXTENDED_B'],
    "Arabic" : ['ARABIC', 'ARABIC_SUPPLEMENT', 'ARABIC_PRESENTATION_FORMS_A', 'ARABIC_PRESENTATION_FORMS_B'],
    "Devanagari" : ['DEVANAGARI', 'VEDIC_EXTENSIONS', 'DEVANAGARI_EXTENDED', 'BENGALI'],
    "Bengali" : ['BENGALI'],
    "Gurmukhi" : ['GURMUKHI'],
    "Tamil" : ['TAMIL'],
    "Telugu" : ['TELUGU'],
    "Malayalam" : ['MALAYALAM'],
    "Myanmar" : ['MYANMAR', 'MYANMAR_EXTENDED_A'],
    "Chinese" : ['CJK_RADICALS_SUPPLEMENT', 'CJK_SYMBOLS_AND_PUNCTUATION', 'CJK_STROKES',\
                 'ENCLOSED_CJK_LETTERS_AND_MONTHS', 'CJK_COMPATIBILITY', 'CJK_UNIFIED_IDEOGRAPHS_EXTENSION_A',\
                 'CJK_UNIFIED_IDEOGRAPHS', 'CJK_COMPATIBILITY_IDEOGRAPHS', 'CJK_COMPATIBILITY_FORMS',\
                 'CJK_UNIFIED_IDEOGRAPHS_EXTENSION_B', 'CJK_UNIFIED_IDEOGRAPHS_EXTENSION_C',\
                 'CJK_UNIFIED_IDEOGRAPHS_EXTENSION_D', 'CJK_COMPATIBILITY_IDEOGRAPHS_SUPPLEMENT']
}

In [213]:
def get_all_borrowed_words(dest, source, expected_unicode, invalid=["Unsupported titles/"]):
    title = f"Category:{dest}_terms_borrowed_from_{source}"
    params = {
        'action': 'query',
        'format': 'json',
        'prop': 'extracts',
        'exintro': True,
        'explaintext': True,
    }

    url = f"https://en.wiktionary.org/w/api.php?action=query&list=categorymembers&cmtitle={title}&cmlimit=max"
    
    borrowed_words = []
    while(True):
        r = requests.get(url,params)
        try:
            for cmember in r.json()['query']['categorymembers']:
                if len(cmember['title']) > 1 and \
                    not cmember['title'].startswith('-') and \
                    not cmember['title'].endswith('-') and \
                    not any(inv in cmember['title'] for inv in invalid) and \
                    not any(c.isdigit() for c in cmember['title']) and \
                    not cmember['title'].isupper() and \
                    unicodeblock.blocks.of(cmember['title'][0]) not in ['BASIC_PUNCTUATION']:
                    borrowed_words.append(cmember['title'])
                    if len(borrowed_words) % 1000 == 0:
                        print(f"Got {len(borrowed_words)}")
            cmcontinue = r.json()['continue']['cmcontinue']
            url = url.split("&cmcontinue")[0]
            url+=f"&cmcontinue={cmcontinue}"
        except KeyError:
            break
    print(f"Done, {len(borrowed_words)}")
    return borrowed_words

In [219]:
'''
def get_all_loans_and_homonyms:
    for each word in borrowed words:
        1. get word page from wiktionary
        2. extract source word from source language
        3. extract homonyms from language!=source language

    return loan_pairs, homonyms
'''

def get_source_word(soup, src_lang, expected_unicode, lst_invalid_words):
    for src_soup in soup.find_all("span", class_="etyl"):
        if src_soup != None and src_soup.find(lambda tag: tag.name == 'a' and src_lang.lower() in tag.text.lower()):
            
            src_soup_final = src_soup.find_next("i")
            src_word = src_soup_final.text if src_soup_final != None else ''
            if any(inv in src_word for inv in lst_invalid_words):
                src_soup_final = src_soup.find_next("strong")
                src_word = src_soup_final.text if src_soup_final != None else ''
                source_word = re.split('[/,]',src_word)[0].strip()
            else:
                source_word = re.split('[/,]',src_word)[0].strip()                
                
            if [unicodeblock.blocks.of(c) in unicode_block_map[expected_unicode] for c in source_word].count(True) < len(source_word)*.8:
                source_word = ''
                
            return source_word
    
    return ''

def get_synonym(borrowed_word, soup, lst_invalid_words):
    all_synonyms = []
    for header in soup.find_all("span", class_ ="nyms synonym"):
        for synonym_header in header.find_all("span", class_ ="Beng"):
            synonym = synonym_header.text
            print(synonym)
            all_synonyms.append([borrowed_word, synonym])
    return all_synonyms
    
    
    
    
def get_homonym(borrowed_word, soup, src_lang, dest_lang, lst_invalid_words):
    all_homonyms = []
    for header in soup.find_all("h2", id=True):
        if dest_lang.lower() in [header.get('id').lower(), header.text.lower()]:
            try:
                for next_header in header.parent.find_all('h3'):
                    if re.match("Etymology [1-9]",next_header.text):
                        # get next sibling and then find_all span etyl?
                        next_sibs = next_header.find_next_siblings()
                        if len(next_sibs) > 0:
                            all_etyms = next_sibs[0].find_all("span", class_="etyl")
                            if len(all_etyms) > 0:
                                etym = all_etyms[0]
                                for tag in etym.find(lambda tag: tag.name == 'a'):

                                    if src_lang.lower() not in tag.text.lower():
                                        homonym_soup = etym.find_next("i")
                                        ff_word = homonym_soup.text if homonym_soup != None else ''
                                        ff_word = re.split('[,;:]',ff_word)[0].strip()
                                        if any(inv in ff_word for inv in lst_invalid_words):
                                            homonym_soup = etym.find_next("strong")
                                            ff_word = homonym_soup.text if homonym_soup != None else ''
                                            ff_word = re.split('[,;:]',ff_word)[0].strip()

                                        meaning = etym.parent.parent.find_all('ol')[0].text.split('\n')[0]
                                        meaning = re.sub(r'\(.+?\)',r'',meaning)
                                        meaning = re.sub(r'\[.+?\]',r'',meaning)
                                        meaning = meaning.replace('  ', ' ')
                                        meaning = re.split('[.;:]',meaning)[0].strip()

                                        all_homonyms.append([borrowed_word,ff_word,tag.text.lower(),meaning])
            except:
                pass
    
    return all_homonyms

            
def get_all_loans_and_homonyms(borrowed_words, dest_lang, src_lang, expected_unicode, lst_invalid_words=['plural', 'not comparable', 'uncountable', ' spelling', '-person possessive'], min_timeout=5, timeout_after_words=200):
    loan_pairs = [] # [[borrowed_word, source_word]]
    homonyms = [] # 
    synonyms = []
    p_bar = tqdm(borrowed_words)
    for i, word in enumerate(p_bar):
        while True:
            try:
                params = {
                    'action': 'query',
                    'format': 'json',
                    'prop': 'extracts',
                    'exintro': True,
                    'explaintext': True,
                }

                url = 'https://en.wiktionary.org/w/rest.php/v1/page/' + word + '/html'

                response = requests.get(url, params, timeout=300)

                soup = BeautifulSoup(response.content, 'html.parser')
                source_word = get_source_word(
                    soup, src_lang, expected_unicode, lst_invalid_words)
                #homonym = get_homonym(
                #    word, soup, src_lang, dest_lang, lst_invalid_words)
                if source_word != '':
                    loan_pairs.append([word, source_word])
                #homonyms.extend(homonym)
                synonym = get_synonym(word, soup, lst_invalid_words)
                synonyms.extend(synonym)
                p_bar.set_description("Processed word: {}".format(word))
                if i > 0 and i % timeout_after_words == 0:
                    sleep_time = (random() * min_timeout) + min_timeout
                    p_bar.set_description("Collected {} word pairs, sleeping for {} seconds".format(i, sleep_time))
                    time.sleep(sleep_time)
                break
            except Exception as e:
                p_bar.set_description("Word: {}, Error: {}, sleeping for 1 minute".format(word, e))
                time.sleep(60)

    return loan_pairs, homonyms, synonyms

In [220]:
for pair in links:
    [dest, src] = pair.split('-')
    print(pair)
    words = get_all_borrowed_words(dest, src, expected_unicodes[pair])
    
    overwrite_loans = None
    overwrite_ff = None
    overwrite_synonym = None
    
    if os.path.exists("results/{}_tatsamas.csv".format(pair)):
        overwrite_loans = input("{}_tatsamas.csv exists. Overwrite existing file? (y/n) ".format(pair))
    else:
        overwrite_loans = "y"
        
#     if os.path.exists("results/{}_homonyms.csv".format(pair)):
#         overwrite_ff = input("{}_homonyms.csv exists. Overwrite existing file? (y/n) ".format(pair))
#     else:
#         overwrite_ff = "y"
    
#     if os.path.exists("results/{}_synonyms.csv".format(pair)):
#         overwrite_synonyms = input("{}_synonyms.csv exists. Overwrite existing file? (y/n) ".format(pair))
#     else:
#         overwrite_synonyms = "y"
    
    if overwrite_loans == "y" or overwrite_ff == "y" or overwrite_synonyms == "y":
        print("Getting loan pairs and homonym: {} candidates".format(len(words)))
        loan_words, homonyms, synonyms = get_all_loans_and_homonyms(words, dest, src, expected_unicodes[pair])
        print(synonyms)
        df_loans = pd.DataFrame(loan_words, columns=['loan_word', 'original_word'])
        #df_homonyms = pd.DataFrame(homonyms, columns=['loan_word', 'original_word', 'other_etymology', 'other_meaning'])
        #df_synonyms = pd.DataFrame(synonyms, columns = ['word', 'synonym'])
        if overwrite_loans == "y":
            df_loans.to_csv("results/{}_tatsamas.csv".format(pair), index=False)
            
        #if overwrite_ff == "y":
            #df_homonyms.to_csv("results/{}_homonyms.csv".format(pair), index=False)
        #if overwrite_synonyms == "y":
            #df_synonyms.to_csv("results/{}_synonyms.csv".format(pair), index=False)
    print(pair, "done\n")
    print()
        

Assamese-Sanskrit
Done, 203
Assamese-Sanskrit_tatsamas.csv exists. Overwrite existing file? (y/n) y
Getting loan pairs and homonym: 203 candidates


Processed word: অক্ষ:   2%|███▋                                                                                                                                                                                      | 4/203 [00:01<01:04,  3.08it/s]

পাশা
অক্ষক্রীড়া
অক্ষরেখা
মেরুরেখা
অক্ষাংশ
অক্ষি
রুদ্রাক্ষ


Processed word: অগ্নি:   2%|████▌                                                                                                                                                                                    | 5/203 [00:01<01:03,  3.11it/s]

আগুন
তেজ
জঠরাগ্নি
অগ্নিকোণ


...

হিমালয়


Processed word: হোয়াদ:  99%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▎ | 309/312 [01:30<00:00,  4.71it/s]

কলব
হৃৎপিন্ড
হিয়া
হার্ট
স্বাদ


Processed word: Category:Bengali semi-learned borrowings from Sanskrit: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 312/312 [01:31<00:00,  3.43it/s]

[['অকাম', 'অকাম্য'], ['অকাম', 'নিষ্কাম'], ['অক্ষ', 'পাশা'], ['অক্ষ', 'অক্ষক্রীড়া'], ['অক্ষ', 'অক্ষরেখা'], ['অক্ষ', 'মেরুরেখা'], ['অক্ষ', 'অক্ষাংশ'], ['অক্ষ', 'অক্ষি'], ['অক্ষ', 'রুদ্রাক্ষ'], ['অক্ষর', 'বর্ণ'], ['অগস্ত্য', 'অগস্ত্যমুনি'], ['অগস্ত্য', 'ক্যানোপাস'], ['অগ্নি', 'আগুন'], ['অগ্নি', 'তেজ'], ['অগ্নি', 'জঠরাগ্নি'], ['অগ্নি', 'অগ্নিকোণ'], ['অঙ্ক', 'সংখ্যা'], ['অঙ্ক', 'আঁক'], ['অঙ্কুশ', 'আঁকশি'], ['অঙ্কুশ', 'ডাঙ্গশ'], ['অঙ্কুশী', 'আঁকশি'], ['অজগর', 'ময়াল'], ['অজগর', 'পাইথন'], ['অত্যাচার', 'জোরাজুরি'], ['অত্যাচার', 'জুলুম'], ['অত্যাচারিত', 'মজলুম'], ['অত্যাচারী', 'জালিম'], ['অত্যাচারী', 'জুলুমকারী'], ['অত্যাচারী', 'জালিম'], ['অনুবাদ', 'তর্জমা'], ['অনুমতি', 'ইজাজত'], ['অপেক্ষা', 'এন্তেজার'], ['অবশ্য', 'আলবৎ'], ['অভদ্র', 'অশিষ্ট'], ['অভদ্র', 'অসভ্য'], ['অভদ্র', 'বেতমিজ'], ['অরবিন্দ', 'পদ্ম'], ['অর্থাৎ', 'মানে'], ['অসভ্য', 'অভদ্র'], ['অসভ্য', 'অমার্জিত'], ['অস্থি', 'হাড়'], ['আকাশ', 'আসমান'], ['আত্মহত্যা', 'আত্মঘাত'], ['আত্মহত্যা', 'আত্মহনন'], ['আত্মহত্যা', 'ইচ্ছামৃত্যু'], ['আত্মহত্

In [128]:
import numpy as np
asm_skt_syn = pd.read_csv('results/Assamese-Sanskrit_synonyms.csv')
ben_skt_syn = pd.read_csv('results/Bengali-Sanskrit_synonyms.csv')
asm_ben_int = pd.read_csv('../Assamese-Bengali-intersection.csv')

In [129]:
asm_list = [w for w in list(asm_skt_syn['word']) if w in list(asm_ben_int['Assamese'])]

In [130]:
ben_list = [w for w in list(ben_skt_syn['word']) if w in list(asm_ben_int['Bengali'])]

In [131]:
asm_skt_syn.loc[asm_skt_syn['word'].isin(asm_list)]

,word,synonym
0,অংক,গণিত
8,অনুবাদ,ভাঙনি
9,অনুবাদ,তৰ্জমা
10,অনুবাদ,পৰিভাষা
11,অনুবাদ,ভাষান্তৰণ
...,...,...
1444,ৰঙা,ৰাতা
1445,ৰঙা,ৰাতুল
1446,ৰঙা,লাল
1450,ৰাং,টিং


In [132]:
ben_skt_syn.loc[ben_skt_syn['word'].isin(ben_list)]

,word,synonym
5,অংশু,অংশুক
6,অংশু,অংশুপট্ট
43,অক্ষ,পাশা
44,অক্ষ,অক্ষক্রীড়া
45,অক্ষ,অক্ষরেখা
...,...,...
1344,হিয়া,হার্ট
1349,হৃদয়,কলব
1350,হৃদয়,হৃৎপিন্ড
1351,হৃদয়,হিয়া


In [137]:
ben_words = ben_skt_syn.loc[ben_skt_syn['word'].isin(ben_list)]
concat_syns = []
for w in ben_words.iterrows():
    word = w[1]['word']
    syn = w[1]['synonym']
    asm_words = [i[1] for i in list(asm_ben_int.loc[asm_ben_int['Bengali'] == word]['Assamese'].items())]
    for aw in asm_words:
        concat_syns.extend([[aw,syn]])
#print(np.array(concat_syns))
df = pd.DataFrame(np.array(concat_syns), columns = ['loan_word','original_word'])
df

,loan_word,original_word
0,আঁহ,অংশুক
1,আঁহ,অংশুপট্ট
2,অক্ষ,পাশা
3,অক্ষ,অক্ষক্রীড়া
4,অক্ষ,অক্ষরেখা
...,...,...
367,হিয়া,হার্ট
368,হিয়া,কলব
369,হিয়া,হৃৎপিন্ড
370,হিয়া,হিয়া


In [139]:
df.to_csv('Assamese-Bengali-Synonyms.csv')

In [208]:
L1 = 'Assamese'
L2 = 'Bengali'
filename = f'../{L1}-{L2}-Cognates.csv'
df = pd.read_csv(filename)
df.reset_index(drop=True).to_csv(filename)

filename = f'../{L1}-{L2}-Synonyms.csv'
df = pd.read_csv(filename)
print(df.shape)
print(df[df["plain Levenshtein"] == 0])
df = df[df["plain Levenshtein"] != 0]
print(df.shape)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df = df.loc[:, ~df.columns.str.contains('^index')]
df.reset_index(drop=True).to_csv(filename)

filename = f'../{L1}-{L2}-Hard-Negatives.csv'
df = pd.read_csv(filename)
print(df.shape)
print(df[df["loan_english"] == df["original_english"]])
df = df[df["loan_english"] != df["original_english"]]
print(df.shape)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df = df.loc[:, ~df.columns.str.contains('^index')]
df.reset_index(drop=True).to_csv(filename)

filename = f'../{L1}-{L2}-Randoms.csv'
df = pd.read_csv(filename)
print(df.shape)
print(df[df["loan_english"] == df["original_english"]])
df = df[df["loan_english"] != df["original_english"]]
print(df.shape)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df = df.loc[:, ~df.columns.str.contains('^index')]
df.reset_index(drop=True).to_csv(filename)

(366, 16)
    loan_word original_word loan_word_epitran original_word_epitran  \
217   বৃশ্চিক       বৃশ্চিক           bɽixsik             bɽiʃt͡ɕik   

    loan_english original_english  Fast Levenshtein Distance Div Maxlen  \
217      Scorpio          Scorpio                              0.444444   

     Dolgo Prime Distance Div Maxlen  Feature Edit Distance Div Maxlen  \
217                         0.285714                          0.050595   

     Hamming Feature Distance Div Maxlen  \
217                             0.053571   

     Weighted Feature Distance Div Maxlen  \
217                                 0.625   

     Partial Hamming Feature Distance Div Maxlen  plain Levenshtein  \
217                                     0.050595                  0   

    loan_unicode original_unicode    label  
217      Bengali          Bengali  synonym  
(365, 16)
(878, 16)
    loan_word original_word loan_word_epitran original_word_epitran  \
0         আঁহ           আঁশ               ã

In [209]:
L1 = 'Bengali'
L2 = 'Assamese'
filename = f'../{L1}-{L2}-Cognates.csv'
df = pd.read_csv(filename)
df.reset_index(drop=True).to_csv(filename)

filename = f'../{L1}-{L2}-Synonyms.csv'
df = pd.read_csv(filename)
print(df.shape)
print(df[df["plain Levenshtein"] == 0])
df = df[df["plain Levenshtein"] != 0]
print(df.shape)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df = df.loc[:, ~df.columns.str.contains('^index')]
df.reset_index(drop=True).to_csv(filename)

filename = f'../{L1}-{L2}-Hard-Negatives.csv'
df = pd.read_csv(filename)
print(df.shape)
print(df[df["loan_english"] == df["original_english"]])
df = df[df["loan_english"] != df["original_english"]]
print(df.shape)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df = df.loc[:, ~df.columns.str.contains('^index')]
df.reset_index(drop=True).to_csv(filename)

filename = f'../{L1}-{L2}-Randoms.csv'
df = pd.read_csv(filename)
print(df.shape)
print(df[df["loan_english"] == df["original_english"]])
df = df[df["loan_english"] != df["original_english"]]
print(df.shape)
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df = df.loc[:, ~df.columns.str.contains('^index')]
df.reset_index(drop=True).to_csv(filename)

(353, 16)
    loan_word original_word loan_word_epitran original_word_epitran  \
211   বৃশ্চিক       বৃশ্চিক         bɽiʃt͡ɕik               bɽixsik   

    loan_english original_english  Fast Levenshtein Distance Div Maxlen  \
211      Scorpio          Scorpio                              0.444444   

     Dolgo Prime Distance Div Maxlen  Feature Edit Distance Div Maxlen  \
211                         0.285714                          0.050595   

     Hamming Feature Distance Div Maxlen  \
211                             0.053571   

     Weighted Feature Distance Div Maxlen  \
211                                 0.625   

     Partial Hamming Feature Distance Div Maxlen  plain Levenshtein  \
211                                     0.050595                  0   

    loan_unicode original_unicode    label  
211      Bengali          Bengali  synonym  
(352, 16)
(813, 16)
    loan_word original_word loan_word_epitran original_word_epitran  \
13       আঁকা          আঁকা               ã

In [189]:
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]
df.to_csv(filename)

In [341]:
ben_skt = pd.read_csv(f'results/Bengali-Sanskrit_tatsamas.csv')
asm_skt = pd.read_csv(f'results/Assamese-Sanskrit_tatsamas.csv')

In [342]:
tatsama = pd.merge(ben_skt, asm_skt, how='inner', on=['loan_word', 'original_word'])
tatsama

,loan_word,original_word
0,অক্ষ,अक्ष
1,অগ্নি,अग्नि
2,অনুবাদ,अनुवाद
3,অনুমতি,अनुमति
4,অশোক,अशोक
5,অস্থি,अस्थि
6,আকাশ,आकाश
7,ইতিহাস,इतिहास
8,উপত্যকা,उपत्यका
9,উপন্যাস,उपन्यास


In [343]:
ben_tatsama = pd.merge(ben_skt, tatsama, how='inner', on=['loan_word'])
ben_tatsama = ben_tatsama.rename(columns={'loan_word' : 'Bengali'})

In [344]:
asm_tatsama = pd.merge(asm_skt, tatsama, how='inner', on=['loan_word'])
asm_tatsama = asm_tatsama.rename(columns={'loan_word' : 'Assamese'})

In [345]:
tatsama = pd.concat([asm_tatsama,ben_tatsama],axis=1)
tatsama = tatsama.loc[:, ~tatsama.columns.str.contains('^original_word')]
tatsama

,Assamese,Bengali
0,অক্ষ,অক্ষ
1,অগ্নি,অগ্নি
2,অনুবাদ,অনুবাদ
3,অনুমতি,অনুমতি
4,অশোক,অশোক
5,অস্থি,অস্থি
6,আকাশ,আকাশ
7,ইতিহাস,ইতিহাস
8,উপত্যকা,উপত্যকা
9,উপন্যাস,উপন্যাস


In [350]:
train = pd.read_csv('production_train_test/Assamese-Bengali/alldata/Assamese-Bengali-train_production_alldata.csv')
test = pd.read_csv('production_train_test/Assamese-Bengali/alldata/Assamese-Bengali-test_production_alldata.csv')


In [351]:
train.loc[train['loan_word'].isin(tatsama['Assamese']) & \
          train['original_word'].isin(tatsama['Bengali']) & \
          train['label_bin'] == 1,'label_bin'] = 4
train.loc[train['label_bin'] == 1,'label_bin'] = 3
train.loc[train['label'] == 'random','label_bin'] = 0
train.loc[train['label'] == 'synonym','label_bin'] = 1
train.loc[train['label'] == 'hard_negative','label_bin'] = 2
train.loc[train['label_bin'] == 3,'label'] = 'tadbhava'
train.loc[train['label_bin'] == 4,'label'] = 'tatsama'
train.to_csv('production_train_test/Assamese-Bengali/alldata/Assamese-Bengali-train_production_alldata-5way.csv')

test.loc[test['loan_word'].isin(tatsama['Assamese']) & \
          test['original_word'].isin(tatsama['Bengali']) & \
          test['label_bin'] == 1,'label_bin'] = 4
test.loc[test['label_bin'] == 1,'label_bin'] = 3
test.loc[test['label'] == 'random','label_bin'] = 0
test.loc[test['label'] == 'synonym','label_bin'] = 1
test.loc[test['label'] == 'hard_negative','label_bin'] = 2
test.loc[test['label_bin'] == 3,'label'] = 'tadbhava'
test.loc[test['label_bin'] == 4,'label'] = 'tatsama'
test.to_csv('production_train_test/Assamese-Bengali/alldata/Assamese-Bengali-test_production_alldata-5way.csv')

In [352]:
train = pd.read_csv('production_train_test/Bengali-Assamese/alldata/Bengali-Assamese-train_production_alldata.csv')
test = pd.read_csv('production_train_test/Bengali-Assamese/alldata/Bengali-Assamese-test_production_alldata.csv')


In [353]:
train.loc[train['loan_word'].isin(tatsama['Assamese']) & \
          train['original_word'].isin(tatsama['Bengali']) & \
          train['label_bin'] == 1,'label_bin'] = 4
train.loc[train['label_bin'] == 1,'label_bin'] = 3
train.loc[train['label'] == 'random','label_bin'] = 0
train.loc[train['label'] == 'synonym','label_bin'] = 1
train.loc[train['label'] == 'hard_negative','label_bin'] = 2
train.loc[train['label_bin'] == 3,'label'] = 'tadbhava'
train.loc[train['label_bin'] == 4,'label'] = 'tatsama'
train.to_csv('production_train_test/Bengali-Assamese/alldata/Bengali-Assamese-train_production_alldata-5way.csv')

test.loc[test['loan_word'].isin(tatsama['Assamese']) & \
          test['original_word'].isin(tatsama['Bengali']) & \
          test['label_bin'] == 1,'label_bin'] = 4
test.loc[test['label_bin'] == 1,'label_bin'] = 3
test.loc[test['label'] == 'random','label_bin'] = 0
test.loc[test['label'] == 'synonym','label_bin'] = 1
test.loc[test['label'] == 'hard_negative','label_bin'] = 2
test.loc[test['label_bin'] == 3,'label'] = 'tadbhava'
test.loc[test['label_bin'] == 4,'label'] = 'tatsama'
test.to_csv('production_train_test/Bengali-Assamese/alldata/Bengali-Assamese-test_production_alldata-5way.csv')